In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
from helpers import (
    REGIJE_FILTER,
    filter_region_year,
)

In [49]:
import locale
from pandas.tseries.offsets import MonthEnd

locale.setlocale(locale.LC_ALL, "sl_SI.utf8")
WIDE_PATH = Path("podatki/wide_df.csv")

df = pd.read_csv(WIDE_PATH,parse_dates=['Datum'])

so2 = filter_region_year(df,"SO2")

so2.dropna(inplace=True)

so2_agg = so2.groupby(['Regija','Datum'])['SO2'].mean().reset_index()

so2_pivot = so2_agg.pivot(index='Datum',columns='Regija',values='SO2')

tmp = so2_pivot[so2_pivot.isna().any(axis=1)]

tmp.reset_index(inplace=True)

tmp = tmp.melt(id_vars='Datum',var_name='Regija',value_name='SO2')

tmp = tmp.loc[tmp['SO2'].isna(),]

pd.concat([so2, tmp]).isna().sum()








Regija    0
Datum     0
SO2       2
dtype: int64